In [1]:
(require esterel-calculus/circuits 
         esterel-calculus/circuits/compiler
         redex/reduction-semantics)

Some possibly helpful laws:

- Posulate 1: `¬( GO ∨ (RES ∧ SEL))` implies all `E'` is `0` and all `kn` are `0`.
- - Correlary 1: This holds for `¬(GO ∨ SEL)`
- - Correlary 2: This holds for `¬(GO ∨ RES)`
- Postulate 2: forall `n` `m`, `m != n`, `¬ (kn ^ km)`, that is at most one `k` wire is selected. (Note: This might be wrong, if GO is given to a selected and resumed pause... this should not be allowed but im not sure...)


In [2]:
(define/ppl start #:no-check
  (
   ;; these come from then othing
   (l0 = GO)
   (lsel = false)
   ;; SEL
   (SEL = (or lsel rsel))
   ;; the synchonizer
   (K0 = (and left0 (and right0 both0)))
   (left0 = (or l0 lem))
   (right0 = (or r0 rem))
   (lem = (and SEL (and RES (not lsel))))
   (rem = (and SEL (and RES (not rsel))))
   (both0 = (or l0 r0))))



(l0 = GO)
(lsel = false)
(SEL = (or lsel rsel))
(K0 = (and left0 (and right0 both0)))
(left0 = (or l0 lem))
(right0 = (or r0 rem))
(lem = (and SEL (and RES (not lsel))))
(rem = (and SEL (and RES (not rsel))))
(both0 = (or l0 r0))


In [28]:
(assert-same 
 #:outputs '(K0 SEL)
 (term start) 
 (term 
  (rename*
   (rename*
         (compile-esterel (par nothing (nts p 0)))
         p-SEL rsel)
   pK0 r0)))

In [4]:
(assert-same (term start) (term ((K0 = r0) (SEL = rsel))))

assert-same: rosette model gave counterexample:
(model
 [pos-GO$2 #f]
 [neg-GO$3 #f]
 [pos-rsel$2 #f]
 [neg-rsel$3 #f]
 [pos-r0$2 #f]
 [neg-r0$3 #t]
 [pos-RES$2 #f]
 [neg-RES$3 #f])
'(((K0 ⊥) (K0 #f)))
'((l0 ⊥)
  (lsel #f)
  (SEL ⊥)
  (K0 ⊥)
  (left0 ⊥)
  (right0 ⊥)
  (lem ⊥)
  (rem ⊥)
  (both0 ⊥)
  (GO ⊥)
  (rsel ⊥)
  (r0 #f)
  (RES ⊥))
'((K0 #f) (SEL ⊥) (GO ⊥) (rsel ⊥) (r0 #f) (RES ⊥))
  context...:
   eval-one-top12
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:510:0: call-with-custodian-shutdown
   /Applications/Racket/racket/collects/racket/private/more-scheme.rkt:148:2: call-with-break-parameterization
   .../more-scheme.rkt:261:28
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:878:5: loop


As we can see, this does not trivially hold. Thus we must reason it ourselves.

In [5]:
(define/ppl translated #:no-check
  (convert-P start))

((+ l0) = (+ GO))
((- l0) = (- GO))
((+ lsel) = false)
((- lsel) = true)
((+ SEL) = (or (+ lsel) (+ rsel)))
((- SEL) = (and (- lsel) (- rsel)))
((+ K0) = (and (+ left0) (and (+ right0) (+ both0))))
((- K0) = (or (- left0) (or (- right0) (- both0))))
((+ left0) = (or (+ l0) (+ lem)))
((- left0) = (and (- l0) (- lem)))
((+ right0) = (or (+ r0) (+ rem)))
((- right0) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (and (+ RES) (- lsel))))
((- lem) = (or (- SEL) (or (- RES) (+ lsel))))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both0) = (or (+ l0) (+ r0)))
((- both0) = (and (- l0) (- r0)))


Step 1: By rules of `and` and `or`

In [6]:
(define/ppl step0 #:no-check
 (propigate/remove* translated
    (+ l0) (- l0) (+ lsel) (- lsel)))

((+ SEL) = (or false (+ rsel)))
((- SEL) = (and true (- rsel)))
((+ K0) = (and (+ left0) (and (+ right0) (+ both0))))
((- K0) = (or (- left0) (or (- right0) (- both0))))
((+ left0) = (or (+ GO) (+ lem)))
((- left0) = (and (- GO) (- lem)))
((+ right0) = (or (+ r0) (+ rem)))
((- right0) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (and (+ RES) true)))
((- lem) = (or (- SEL) (or (- RES) false)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both0) = (or (+ GO) (+ r0)))
((- both0) = (and (- GO) (- r0)))


In [7]:
(define/ppl step1
  replace* step0 
  [(or false (+ rsel)) (+ rsel)]
  [(and true (- rsel)) (- rsel)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left0) (and (+ right0) (+ both0))))
((- K0) = (or (- left0) (or (- right0) (- both0))))
((+ left0) = (or (+ GO) (+ lem)))
((- left0) = (and (- GO) (- lem)))
((+ right0) = (or (+ r0) (+ rem)))
((- right0) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (and (+ RES) true)))
((- lem) = (or (- SEL) (or (- RES) false)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both0) = (or (+ GO) (+ r0)))
((- both0) = (and (- GO) (- r0)))


In [8]:
(define/ppl step-before-case
  replace*
  step1
  [(and (+ RES) true) (+ RES)]
  [(or (- RES) false) (- RES)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left0) (and (+ right0) (+ both0))))
((- K0) = (or (- left0) (or (- right0) (- both0))))
((+ left0) = (or (+ GO) (+ lem)))
((- left0) = (and (- GO) (- lem)))
((+ right0) = (or (+ r0) (+ rem)))
((- right0) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both0) = (or (+ GO) (+ r0)))
((- both0) = (and (- GO) (- r0)))


In [9]:
(define/ppl simp1
  propigate/remove*
  step-before-case
  (+ both0) (- both0) 
  (+ left0) (- left0)
  (+ right0) (- right0))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0)
 =
 (and (or (+ GO) (+ lem)) (and (or (+ r0) (+ rem)) (or (+ GO) (+ r0)))))
((- K0)
 =
 (or (and (- GO) (- lem)) (or (and (- r0) (- rem)) (and (- GO) (- r0)))))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))


In [10]:
(define/ppl simp2
  propigate*
  simp1
  (+ SEL) (- SEL))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0)
 =
 (and (or (+ GO) (+ lem)) (and (or (+ r0) (+ rem)) (or (+ GO) (+ r0)))))
((- K0)
 =
 (or (and (- GO) (- lem)) (or (and (- r0) (- rem)) (and (- GO) (- r0)))))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = (and (+ rsel) (and (+ RES) (- rsel))))
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


In [11]:
(define/ppl simp3
  replace*
  simp2
  [(and (or (+ r0) (+ rem)) (or (+ GO) (+ r0)))
   (or (+ r0) (and (+ rem) (+ GO)))]
  [(or (and (- r0) (- rem)) (and (- GO) (- r0)))
   (and (- r0) (or (- rem) (- GO)))]
  [(and (+ rsel) (and (+ RES) (- rsel)))
   false])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (or (+ GO) (+ lem)) (or (+ r0) (and (+ rem) (+ GO)))))
((- K0) = (or (and (- GO) (- lem)) (and (- r0) (or (- rem) (- GO)))))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = false)
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


In [12]:
(define/ppl simp4
  propigate/remove*
  simp3
  (+ lem) (- lem)
  (+ rem) (- rem))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0)
 =
 (and (or (+ GO) (and (+ rsel) (+ RES))) (or (+ r0) (and false (+ GO)))))
((- K0)
 =
 (or (and (- GO) (or (- rsel) (- RES)))
     (and (- r0) (or (or (- rsel) (or (- RES) (+ rsel))) (- GO)))))


In [13]:
(define/ppl simp5
  replace* simp4
  [(or (+ r0) (and false (+ GO))) (+ r0)]
  [(or (or (- rsel) (or (- RES) (+ rsel))) (- GO))
   (or (or (- rsel) (- RES)) (or (+ rsel) (- GO)))])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (or (+ GO) (and (+ rsel) (+ RES))) (+ r0)))
((- K0)
 =
 (or (and (- GO) (or (- rsel) (- RES)))
     (and (- r0) (or (or (- rsel) (- RES)) (or (+ rsel) (- GO))))))


In [14]:
(define/ppl simp6
  replace* simp5
  [(and (or (+ GO) (and (+ rsel) (+ RES))) (+ r0))
   (and (+ r0) (or (+ GO) (and (+ rsel) (+ RES))))])
   

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ r0) (or (+ GO) (and (+ rsel) (+ RES)))))
((- K0)
 =
 (or (and (- GO) (or (- rsel) (- RES)))
     (and (- r0) (or (or (- rsel) (- RES)) (or (+ rsel) (- GO))))))


In [15]:
(define/ppl simp7
  replace* simp6
  [(or (and (- GO) (or (- rsel) (- RES)))
       (and (- r0) (or (or (- rsel) (- RES)) (or (+ rsel) (- GO)))))
   ;; CNF
   (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel))))
        (and (or (- GO) (- r0)) 
             (or (- rsel) (or (- RES) (- r0)))))])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ r0) (or (+ GO) (and (+ rsel) (+ RES)))))
((- K0)
 =
 (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel))))
      (and (or (- GO) (- r0)) (or (- rsel) (or (- RES) (- r0))))))


In [16]:
(define/ppl simp8
  replace* simp7
  [(and (or (- GO) (- r0)) 
        (or (- rsel) (or (- RES) (- r0))))
   (or (- r0)
       (and (- GO) (or (- rsel) (- RES))))])
   

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ r0) (or (+ GO) (and (+ rsel) (+ RES)))))
((- K0)
 =
 (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel))))
      (or (- r0) (and (- GO) (or (- rsel) (- RES))))))


In [17]:
(define/ppl expanded #:no-check
 (((+ SEL) = (+ rsel))
  ((- SEL) = (- rsel))
  ((+ is-res) = (and (+ rsel) (+ RES)))
  ((- is-res) = (or (- rsel) (- RES)))
  ((+ active) = (or (+ GO) (+ is-res)))
  ((- active) = (and (- GO) (- is-res)))
  ((+ K0) = (and (+ r0) (+ active)))
  ((- K0)
   =
   (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel))))
        (or (- r0) (- active))))))
(assert-same (term expanded) (term simp8))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ is-res) = (and (+ rsel) (+ RES)))
((- is-res) = (or (- rsel) (- RES)))
((+ active) = (or (+ GO) (+ is-res)))
((- active) = (and (- GO) (- is-res)))
((+ K0) = (and (+ r0) (+ active)))
((- K0)
 =
 (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel)))) (or (- r0) (- active))))


Hypothesis 1: `(or (- SEL) (+ SEL))` is true for all circuits, (essientially, SEL is always defined). This gives us:

```
((+ K0) = (and (+ r0) (+ active)))
((- K0) = (or (- r0) (- active))))
 ```

or, in the construtive formulation

```
(K0 = (and r0 active))
```.

Theorem 1: `K0 => active` for all circuits: that is unless something is being resumed or initially activatd it cannot K0. This is follows from Posulate 1. 

Thus, if Hypothesis 1 holds, then K0 = R0, and we are done.

Hypothesis 1 can be proven trivally via induction on terms. However it requires the assumption that no register ever contains an undefined value. I believe is a sound assumption because if a register contains an undefined value it's input wire must have been undefined in the previous instant, thus we would have already errored and never reached that state.

Thus we can apply the following transformation.

In [18]:
(define/ppl applying-thrms #:no-check
  (replace* expanded
   [(and (+ r0) (+ active)) (+ r0)]
   [(or (- rsel) (+ rsel)) true]))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ is-res) = (and (+ rsel) (+ RES)))
((- is-res) = (or (- rsel) (- RES)))
((+ active) = (or (+ GO) (+ is-res)))
((- active) = (and (- GO) (- is-res)))
((+ K0) = (+ r0))
((- K0) = (and (or (- GO) (or (- RES) true)) (or (- r0) (- active))))


In [19]:
(define/ppl simplifying
  replace* applying-thrms
  [(or (- GO) (or (- RES) true)) true]
  [(and true (or (- r0) (- active)))
   (or (- r0) (- active))])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ is-res) = (and (+ rsel) (+ RES)))
((- is-res) = (or (- rsel) (- RES)))
((+ active) = (or (+ GO) (+ is-res)))
((- active) = (and (- GO) (- is-res)))
((+ K0) = (+ r0))
((- K0) = (or (- r0) (- active)))


In [20]:
(define/ppl subst
  propigate/remove* simplifying
  (+ is-res) (- is-res)
  (+ active) (- active))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (+ r0))
((- K0) = (or (- r0) (and (- GO) (or (- rsel) (- RES)))))


`(and (- GO) (or (- rsel) (- RES)))` is the `+` part of `¬( GO ∨ (RES ∧ SEL))`:

In [21]:
(term (convert-p + (not (or GO (and RES SEL)))))

'(and (- GO) (or (- RES) (- SEL)))

by Posulate 1, if `-k0` is false then `¬( GO ∨ (RES ∧ SEL))` cannot be true, as that would have set `-k0` to `1`. Thus the `+` part of `¬( GO ∨ (RES ∧ SEL))` must also be false. Thus `(or (- r0) (and (- GO) (or (- rsel) (- RES))))` is equal to `-r0`, as it is false if `-r0` is, and true if `-r0` is.

In [22]:
(define/ppl final #:no-check
  (replace* subst
   [(or (- r0) (and (- GO) (or (- rsel) (- RES)))) (- r0)]))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (+ r0))
((- K0) = (- r0))


We can follow the same chain of reasoning for any `kn` in `done`, thus we are done.

# QED